In [1]:
import json
import csv
import pandas as pd
import numpy as np

In [2]:
def json2csv(json_name, csv_name, key):
    with open(json_name) as json_file:
        data = json.load(json_file)
    transaction_data = data[key]
    data_file = open(csv_name, 'w') 
    csv_writer = csv.writer(data_file)
    count = 0
    for tran in transaction_data:
        if count == 0: 
            # Writing headers of CSV file 
            header = tran.keys()
            csv_writer.writerow(header) 
            count += 1
        # Writing data of CSV file 
        csv_writer.writerow(tran.values())
    data_file.close() 

In [3]:
json2csv('allfidata_fish.json', 'bank_details.csv', 'data')

In [4]:
df = pd.read_csv('bank_details.csv')

In [5]:
df

,linkReferenceNumber,maskedAccountNumber,bank,Summary,Profile,Transactions
0,11fabe78-0a59-4c71-8fdb-57e54779d236,XXXXXXXXXX5055,FinShareBankServer,"{'currentBalance': '101666.33', 'currency': 'I...","{'Holders': {'type': 'SINGLE', 'Holder': [{'na...","{'startDate': '2019-07-11', 'endDate': '2019-0..."
1,4797daf8-75df-4359-8e61-a710d7e114fa,XXXXXXXXXX5055,FinShareBankServer,"{'branch': 'Jayanagar 4th Block', 'openingDate...","{'Holders': {'type': 'SINGLE', 'Holder': [{'na...","{'startDate': '2019-07-11', 'endDate': '2020-0..."
2,d68d3aa8-4d6d-4365-afd1-4f0822eb5d7e,XXXXXXXXXX0256,FinShareBankServer,"{'branch': 'JayaNagar 4th Block', 'openingDate...","{'Holders': {'type': 'SINGLE', 'Holder': [{'na...","{'startDate': '2019-07-11', 'endDate': '2020-0..."


In [6]:
df["Summary"][1]

"{'branch': 'Jayanagar 4th Block', 'openingDate': '2019-12-22', 'ifsc': 'ICIC0001124', 'accountType': 'FIXED', 'maturityDate': '2021-01-16', 'maturityAmount': '105622.00', 'description': '390 Days Cumulative Term Deposit ', 'interestPayout': 'OnMaturity', 'interestRate': '5.15', 'principalAmount': '100000.00', 'tenureDays': '390', 'tenureMonths': '0', 'tenureYears': '0', 'interestComputation': 'COMPOUND', 'compoundingFrequency': 'QUARTERLY', 'interestPeriodicPayoutAmount': '0', 'interestOnMaturity': '5622', 'currentValue': '100000.00'}"

In [13]:
bank_name, account_type, account_no, principalAmount, maturityAmount, openingDate, maturityDate, interestRate, description  = [], [], [], [], [], [], [], [], []
for i in range(df.shape[0]):
    stri1 = df.at[i,"Summary"].replace("{","")
    stri1 = stri1.replace("'",'')
    #print(stri1)
    li = stri1.split(',')
    #print(li)
    type_ac = li[3].split(':')
    #print(type_ac)
    type_account1 = type_ac[1].strip() #Account type
    #print(type_account1)
    if type_account1 == "FIXED":
        #print("in")
        bank_name1 = df.at[i,"bank"] #Bank name
        account_no1 = df.at[i,"maskedAccountNumber"] #Account no
        opening_da = li[1].split(":")
        opening_date1 = opening_da[1].strip() #Opening Date
        maturity_da = li[4].split(':')
        maturity_date1 = maturity_da[1].strip() #Maturity date
        maturity_am = li[5].split(":")
        maturity_amount1 = maturity_am[1].strip() #Maturity Amount
        desc = li[6].split(":")
        description1 = desc[1].strip() #Description
        int_rate = li[8].split(":")
        interest_rate1 = int_rate[1].strip() #Interest Rate
        principal_am = li[9].split(':')
        principal_amount1 = principal_am[1].strip() #Principal Amount
        bank_name.append(bank_name1)
        account_type.append(type_account1)
        account_no.append(account_no1)
        openingDate.append(opening_date1)
        maturityDate.append(maturity_date1)
        principalAmount.append(principal_amount1)
        maturityAmount.append(maturity_amount1)
        interestRate.append(interest_rate1)
        description.append(description1)
        #print(bank_name, account_type, account_no, principalAmount, maturityAmount, openingDate, maturityDate, interestRate)
        #Account Type
        #print (bank_name1, current_balance1, account_no1, type_account1)



In [16]:
df_out = pd.DataFrame({"bankName": bank_name, "accountNo": account_no, "typeAccount": account_type, 
                      "principalAmount": principalAmount, "maturityAmount": maturityAmount, "openingDate":openingDate, "maturityDate": maturityDate, 
                       "interestRate": interestRate, "description": description})
df_out

,bankName,accountNo,typeAccount,principalAmount,maturityAmount,openingDate,maturityDate,interestRate,description
0,FinShareBankServer,XXXXXXXXXX5055,FIXED,100000.00,105622.00,2019-12-22,2021-01-16,5.15,390 Days Cumulative Term Deposit


In [17]:
df_out.to_json("../web-app/server/src/fdDetails.json", orient = 'records')